In [80]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain, LLMChain, APIChain
from langchain import PromptTemplate
from langchain.prompts import StringPromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import MetaphorSearchAPIWrapper
from langchain.agents import Agent, Tool, initialize_agent, AgentType, AgentExecutor, tool, AgentOutputParser
from metaphor_python import Metaphor
from dotenv import load_dotenv
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from typing import List, Dict, Any, Optional, Union
import re

import os

In [81]:
load_dotenv('.env')

openai_api_key = os.getenv('OPENAI_API_KEY')
metaphor_api_key = os.getenv('METAPHOR_API_KEY')

metaphor_client = Metaphor(metaphor_api_key)

@tool
def search(query: str):
    """Call search engine with a query."""
    return metaphor_client.search(query, use_autoprompt=True, num_results=3)

@tool
def get_contents(ids: List[str]):
    """Get contents of a webpage.
    
    The ids passed in should be a list of ids as fetched from `search`.
    """
    return metaphor_client.get_contents(ids)

@tool
def find_similar(url: str):
    """Get search results similar to a given URL.
    
    The url passed in should be a URL returned from `search`
    """
    return metaphor_client.find_similar(url, num_results=3)

In [82]:
metaphor_search = MetaphorSearchAPIWrapper(metaphor_api_key=metaphor_api_key)
llm = ChatOpenAI(model='gpt-3.5-turbo', openai_api_key=openai_api_key)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    search, 
    get_contents, 
    find_similar,
    Tool(
        name='calculator',
        func=llm_math_chain.run,
        description='Calculate information from search results'
    )
]

In [83]:
from langchain.agents import OpenAIFunctionsAgent
# template = """ You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

# Question: {input}
# {agent_scratchpad}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     '''You are a venture capitalist conducting due dilligence on a startup. Find information about the startup and answer the following questions:
        1. Analyze the startup's business model
        2. Compare the startup's competitors
        3. Analyze the startup's market
        4. Evaluate the startup's growth potential
        5. Evaluate the startup's team
     '''),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [84]:
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [85]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [86]:
# agent = initialize_agent(tools, llm=llm, agent=AgentType.OPENAI_FUNCTIONS)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [87]:
agent_output = agent_executor.invoke({"input": "Evaluate Ramp's potential"})
agent_output['output']



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'Ramp startup'}`


Title: Home - Ramp Armor
URL: https://www.ramparmor.com/
ID: xfXNjLpvtoJu7LEYmR6WJw
Score: 0.21130944788455963
Published Date: 2021-04-12
Author: Mark Waters Emerica Promotions Manager; Skater; Photographer
Extract: None

Title: Aqcess Launch | Transoft Solutions
URL: https://www.transoftsolutions.com/aqcesslaunch/
ID: fD4xE7mdZSI-vK5TqofoEQ
Score: 0.2053244262933731
Published Date: 2020-06-30
Author: None
Extract: None

Title: Home - Peaked Pallets
URL: https://peakedpallets.com/
ID: ZUIF8oPSi7YO7zAuHuwKuw
Score: 0.20061898231506348
Published Date: 2019-04-20
Author: None
Extract: None

Title: OONEEPOD
URL: https://www.ooneepod.com/
ID: TSUKuJavdNf2jrNCJXy-TQ
Score: 0.1882365345954895
Published Date: 2022-01-01
Author: None
Extract: None

Title: Advanced Paving Technologies
URL: http://www.adpavtec.com/
ID: i_bg9bdU67xPVWAE4jwqAg
Score: 0.18162432312965393
Published Date: 2006-01-01
Author: 

"Ramp is a startup that specializes in building ramps for skateboarding and other extreme sports. Their website, Ramp Armor, showcases their products and services. Here is an analysis of Ramp's potential:\n\n1. Business Model:\n   - Ramp offers high-performance surface materials for skatepark builds.\n   - They collaborate with experienced skatepark builders worldwide.\n   - Their surfaces are affordable and designed to withstand different climates.\n   - Ramp's surfaces are known for their durability and long-lasting performance.\n\n2. Competitors:\n   - While there are several skatepark builders and ramp suppliers in the market, Ramp differentiates itself through its high-performance surface materials.\n   - Some competitors may offer similar services, but Ramp's focus on quality, longevity, and affordability sets them apart.\n\n3. Market Analysis:\n   - The skateboarding and extreme sports market has been growing steadily in recent years.\n   - There is a demand for well-designed an

In [88]:
print(agent_output['output'])

Ramp is a startup that specializes in building ramps for skateboarding and other extreme sports. Their website, Ramp Armor, showcases their products and services. Here is an analysis of Ramp's potential:

1. Business Model:
   - Ramp offers high-performance surface materials for skatepark builds.
   - They collaborate with experienced skatepark builders worldwide.
   - Their surfaces are affordable and designed to withstand different climates.
   - Ramp's surfaces are known for their durability and long-lasting performance.

2. Competitors:
   - While there are several skatepark builders and ramp suppliers in the market, Ramp differentiates itself through its high-performance surface materials.
   - Some competitors may offer similar services, but Ramp's focus on quality, longevity, and affordability sets them apart.

3. Market Analysis:
   - The skateboarding and extreme sports market has been growing steadily in recent years.
   - There is a demand for well-designed and durable skate